In [1]:
import numpy as np
import os
import cv2 as cv

In [2]:
path_list = os.listdir("./data")
path_list

['in_01.png',
 'in_02.png',
 'in_03.png',
 'in_05.png',
 'in_06.png',
 'in_08.png',
 'in_09.png',
 'in_11.png',
 'in_15.png',
 'in_20.png',
 'in_24.png',
 'in_29.png',
 'in_50.png',
 'in_51.png',
 'in_56.png',
 'res_01.png',
 'res_02.png',
 'res_03.png',
 'res_05.png',
 'res_06.png',
 'res_08.png',
 'res_09.png',
 'res_11.png',
 'res_15.png',
 'res_20.png',
 'res_24.png',
 'res_29.png',
 'res_50.png',
 'res_51.png',
 'res_56.png',
 'tar_01.png',
 'tar_02.png',
 'tar_03.png',
 'tar_05.png',
 'tar_06.png',
 'tar_08.png',
 'tar_09.png',
 'tar_11.png',
 'tar_15.png',
 'tar_20.png',
 'tar_24.png',
 'tar_29.png',
 'tar_50.png',
 'tar_51.png',
 'tar_56.png']

In [3]:
targets, sources, results = [], [], []

In [4]:
{targets.append(path) if path.startswith("tar") else sources.append(path) if path.startswith("in") else results.append(
    path) for path in path_list}
targets, sources, results

(['tar_01.png',
  'tar_02.png',
  'tar_03.png',
  'tar_05.png',
  'tar_06.png',
  'tar_08.png',
  'tar_09.png',
  'tar_11.png',
  'tar_15.png',
  'tar_20.png',
  'tar_24.png',
  'tar_29.png',
  'tar_50.png',
  'tar_51.png',
  'tar_56.png'],
 ['in_01.png',
  'in_02.png',
  'in_03.png',
  'in_05.png',
  'in_06.png',
  'in_08.png',
  'in_09.png',
  'in_11.png',
  'in_15.png',
  'in_20.png',
  'in_24.png',
  'in_29.png',
  'in_50.png',
  'in_51.png',
  'in_56.png'],
 ['res_01.png',
  'res_02.png',
  'res_03.png',
  'res_05.png',
  'res_06.png',
  'res_08.png',
  'res_09.png',
  'res_11.png',
  'res_15.png',
  'res_20.png',
  'res_24.png',
  'res_29.png',
  'res_50.png',
  'res_51.png',
  'res_56.png'])

In [5]:
target_list, source_list, result_list = [], [], []
for i in range(len(targets)):
    # target img
    target_path = os.path.join("./data", targets[i])
    target_img = cv.imread(target_path)
    target_img = cv.cvtColor(target_img, cv.COLOR_BGR2LAB).astype("float32")
    target_list.append(target_img)
    # source img
    source_path = os.path.join("./data", sources[i])
    source_img = cv.imread(source_path)
    source_img = cv.cvtColor(source_img, cv.COLOR_BGR2LAB).astype("float32")
    source_list.append(source_img)
    # result img
    result_path = os.path.join("./data", results[i])
    result_img = cv.imread(result_path)
    result_img = cv.cvtColor(result_img, cv.COLOR_BGR2LAB).astype("float32")
    result_list.append(result_img)

In [6]:
# returns the Mean and Std for LAB channels
def image_stats(x):
    l, a, b = cv.split(x)
    lMean, lStd = l.mean(), l.std()
    aMean, aStd = a.mean(), a.std()
    bMean, bStd = b.mean(), b.std()

    return (lMean, lStd, aMean, aStd, bMean, bStd)

In [7]:
# transfer the color of images
def transfer_img(source_img, target_img):
    (lMeanSrc, lStdSrc, aMeanSrc, aStdSrc, bMeanSrc, bStdSrc) = image_stats(source_img)
    (lMeanTar, lStdTar, aMeanTar, aStdTar, bMeanTar, bStdTar) = image_stats(target_img)

    l, a, b = cv.split(source_img)
    l -= lMeanSrc
    a -= aMeanSrc
    b -= bMeanSrc

    l *= (lStdSrc / lStdTar)
    a *= (aStdSrc / aStdTar)
    b *= (bStdSrc / bStdTar)

    l += lMeanTar
    a += aMeanTar
    b += bMeanTar

    l = np.clip(l, 0, 255)
    a = np.clip(a, 0, 255)
    b = np.clip(b, 0, 255)

    transfer = cv.merge([l, a, b])
    transfer = cv.cvtColor(transfer.astype("uint8"), cv.COLOR_LAB2BGR)
    return transfer

In [8]:
# transferring the colors and save in a directory
for n in range(len(source_list)):
    source_img, target_img = source_list[n], target_list[n]
    transfer = transfer_img(source_img, target_img)
    cv.imwrite(f"./result/{results[n]}", transfer)

In [9]:
# SSD
def ssd(X, Y):
    return np.sum((X - Y) ** 2)

In [10]:
# NCC
def cor(X, Y):
    return np.sum(X * Y) / np.sum(X) / np.sum(Y)

In [11]:
# SSD and NCC => SSD x NCC
def ssd_and_cor(X, Y):
    return ssd(X, Y) * cor(X, Y)

splitting image

In [12]:
# dividing function for images
def split(img, m, n):
    height, width = img.shape[0], img.shape[1]
    h = height // m
    w = width // n
    imgs = []
    for i in range(0, height // m * m, h):
        for j in range(0, width // n * n, w):
            imgs.append(np.clip(img[i:i + h, j:j + w, :], 0, 255))
    return imgs

In [13]:
# concatenate function
def concat_vh(list_2d):
    arr = [cv.hconcat(np.array(list_h)) for list_h in list_2d]
    return cv.vconcat(np.array(arr))

In [14]:
# a function returns images for different number of subimages and methods
def func(sImg, tImg, method):
    diffs = []
    for i in range(2, 6):
        for j in range(2, 6):
            sImgs = split(sImg, i, j)
            tImgs = split(tImg, i, j)
            for k in range(i * j):
                diffs.append(method(sImgs[k], tImgs[k]))
    imgs = {}
    for i in range(2, 6):
        for j in range(2, 6):
            sImgs = split(sImg, i, j)
            tImgs = split(tImg, i, j)
            for k in range(i * j):
                if (method(sImgs[k], tImgs[k]) < np.mean(diffs)):
                    sImgs[k] = transfer_img(sImgs[k], tImgs[k])
            Imgs2d = []
            for k in range(i):
                Imgs = []
                for l in range(j):
                    Imgs.append(sImgs[k * len(sImgs) // i + l])
                Imgs2d.append(Imgs)

            imgs[(i, j)] = concat_vh(Imgs2d)
    return imgs

In [15]:
for i in range(len(source_list)):
    source_img, target_img = source_list[i], target_list[i]

    minHeight = min(source_img.shape[0], target_img.shape[0])
    minWidth = min(source_img.shape[1], target_img.shape[1])

    simg = cv.resize(source_img, (minWidth, minHeight))
    timg = cv.resize(target_img, (minWidth, minHeight))

    imgs_ssd = func(simg, timg, ssd)
    imgs_cor = func(simg, timg, cor)
    imgs_ssd_and_cor = func(simg, timg, ssd_and_cor)

    for k in imgs_ssd.keys():
        cv.imwrite(f"./results_part2/ssd/{results[i][:-4]}/{k}.png", imgs_ssd[k])
        cv.imwrite(f"./results_part2/cor/{results[i][:-4]}/{k}.png", imgs_cor[k])
        cv.imwrite(f"./results_part2/ssd_and_cor/{results[i][:-4]}/{k}.png", imgs_ssd_and_cor[k])

C:\Users\GorkemKola\AppData\Local\Temp\ipykernel_9020\1756009796.py:12: RuntimeWarning: divide by zero encountered in float_scalars
  a *= (aStdSrc / aStdTar)
C:\Users\GorkemKola\AppData\Local\Temp\ipykernel_9020\1756009796.py:13: RuntimeWarning: divide by zero encountered in float_scalars
  b *= (bStdSrc / bStdTar)
C:\Users\GorkemKola\AppData\Local\Temp\ipykernel_9020\1756009796.py:11: RuntimeWarning: divide by zero encountered in float_scalars
  l *= (lStdSrc / lStdTar)
